In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!git clone https://github.com/Hua0113/DCAP
%cd DCAP
!pip freeze > requirements.txt

Cloning into 'DCAP'...
remote: Enumerating objects: 74, done.
remote: Total 74 (delta 0), reused 0 (delta 0), pack-reused 74
Unpacking objects: 100% (74/74), done.
/content/DCAP


In [3]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.8 MB 5.0 MB/s 
Processing /colabtools/dist/google-colab-1.0.0.tar.gz
ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: '/colabtools/dist/google-colab-1.0.0.tar.gz'



In [4]:
!pip install tensorflow==1.14

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 109.3 MB 1.1 MB/s 
     |████████████████████████████████| 50 kB 7.0 MB/s 
     |████████████████████████████████| 488 kB 69.6 MB/s 
     |████████████████████████████████| 3.1 MB 66.8 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.8.2+zzzcolab20220719082949
    Uninstalling tensorflow-2.8.2+zzzcolab20220719082949:
      Successfully uninstalled tensorflow-2.8.2+zzzcolab20220719082949
ERROR: pip's dependency resolver does not currently t

In [5]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 

import numpy as np  
import matplotlib.pyplot as plt  
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

Instructions for updating:
non-resource variables are not supported in the long term


/usr/local/lib/python3.7/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:544: FutureWa

In [32]:
with open(r"/content/drive/MyDrive/U-BRITE_Hackathon/dataset/BRCA/brcatest_go.csv", 'r') as f:
    data = pd.read_csv(f)

print(data.shape)
tcga_input=np.transpose(data)
print(tcga_input.shape)

learning_rate = 0.01
training_epochs = 1
batch_size = 10
display_step = 1
examples_to_show = 10

dropout=0.1
n_input = 25 # 60779
scale = 0.0001
# tf Graph input (only pictures)
X = tf.placeholder("float", [None, n_input])


n_hidden_1 = 500 # 
n_hidden_2 = 200 # 

weights = {
    'encoder_h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'encoder_h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'decoder_h1': tf.Variable(tf.random_normal([n_hidden_2, n_hidden_1])),
    'decoder_h2': tf.Variable(tf.random_normal([n_hidden_1, n_input])),
}
biases = {
    'encoder_b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'encoder_b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'decoder_b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'decoder_b2': tf.Variable(tf.random_normal([n_input])),
}


def encoder(x):
    layer_1 = tf.nn.tanh(tf.add(tf.matmul(x, weights['encoder_h1']),
                                   biases['encoder_b1']))
    layer_2 = tf.nn.tanh(tf.add(tf.matmul(layer_1, weights['encoder_h2']),
                                   biases['encoder_b2']))
    return layer_2



def decoder(x):
    layer_1 = tf.nn.tanh(tf.add(tf.matmul(x, weights['decoder_h1']),
                                   biases['decoder_b1']))
    layer_2 = tf.nn.tanh(tf.add(tf.matmul(layer_1, weights['decoder_h2']),
                                   biases['decoder_b2']))
    return layer_2


fc_1 = tf.layers.dense(inputs=X, units=n_hidden_1,
                       kernel_regularizer= tf.keras.regularizers.l2(scale))  #   tf.contrib.layers.l2_regularizer(scale=scale))
fc_1_out = tf.nn.tanh(fc_1)
fc_1_dropout = tf.layers.dropout(inputs=fc_1_out, rate=dropout)

fc_2 = tf.layers.dense(inputs = fc_1_dropout, units = n_hidden_2, kernel_regularizer=tf.keras.regularizers.l2(scale)) #, kernel_regularizer= tf.contrib.layers.l2_regularizer(scale=scale))
fc_2_out = tf.nn.tanh(fc_2)
encoder_op = tf.layers.dropout(inputs=fc_2_out, rate=dropout)

fc_3 = tf.layers.dense(inputs = encoder_op, units = n_hidden_1, kernel_regularizer=tf.keras.regularizers.l2(scale)) #, kernel_regularizer= tf.contrib.layers.l2_regularizer(scale=scale))

fc_3_out = tf.nn.tanh(fc_3)
fc_3_dropout = tf.layers.dropout(inputs=fc_3_out, rate=dropout)

decoder_op = tf.layers.dense(inputs=fc_3_dropout, units=n_input)


y_pred = decoder_op
y_true = X


cost = tf.reduce_mean(tf.pow(y_true - y_pred, 2))#+lossL 
l2_loss = tf.losses.get_regularization_loss()
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost+l2_loss)

with tf.Session() as sess:
    # tf.initialize_all_variables() no long valid from
    if int((tf.__version__).split('.')[1]) < 12 and int((tf.__version__).split('.')[0]) < 1:
        init = tf.initialize_all_variables()
    else:
        init = tf.global_variables_initializer()
    sess.run(init)
  
    total_batch = int(len(tcga_input)/batch_size) 
    for epoch in range(training_epochs):
        for i in range(total_batch):
            # tch_xs, batch_ys = mnist.train.next_batch(batch_size)  # max(x) = 1, min(x) = 0
            batch_xs = tcga_input[((i)*batch_size):((i+1)*batch_size)]

            # Run optimization op (backprop) and cost op (to get loss value)
            c = sess.run([optimizer, cost], feed_dict={X: batch_xs})
            # _ = sess.run([optimizer, cost], feed_dict={X: batch_xs})
        if epoch % display_step == 0:
          print()
            #print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(c))
        if epoch == training_epochs - 1:
                fea_output = sess.run([encoder_op], feed_dict={X: tcga_input})
                # print(fea_output)
                print(np.array(fea_output).shape)
                np.savetxt(r'fea.csv', np.array(fea_output[0]), delimiter=',')

(25, 10)
(10, 25)



(1, 10, 200)
Optimization Finished!


In [34]:
import pandas as pd

pd.read_csv('fea.csv')

,3.905583918094635010e-02,-1.120344102382659912e-01,6.781383603811264038e-02,-1.006475389003753662e-01,-7.184841036796569824e-01,-4.465349018573760986e-01,-7.437600493431091309e-01,4.270775914192199707e-01,-2.683780342340469360e-02,9.707919955253601074e-01,...,-9.571436047554016113e-01,-6.874125003814697266e-01,-8.347727656364440918e-01,7.119297981262207031e-01,-2.130798548460006714e-01,9.058490395545959473e-01,7.661555893719196320e-03,3.857931792736053467e-01,3.990606665611267090e-01,5.086650848388671875e-01
0,0.949649,0.479984,-0.386437,0.496759,0.675644,0.590998,-0.915888,0.253031,0.203126,0.341104,...,0.381077,-0.103251,-0.729361,0.834897,0.083487,0.301211,0.356741,0.764055,-0.112118,0.541362
1,-0.373962,0.018485,-0.534157,-0.035274,0.100847,0.002324,0.323503,0.382957,-0.054797,0.146521,...,-0.583507,-0.355580,-0.507217,-0.177519,0.440569,0.405455,-0.215503,0.327706,0.648900,0.231457
2,-0.552905,-0.210467,-0.172624,-0.269681,0.362824,-0.279777,0.568637,0.401046,-0.330928,-0.449040,...,-0.178288,-0.009785,-0.278470,-0.272515,0.626785,0.381059,0.038159,0.244361,-0.205838,0.356090
3,0.507259,0.187694,-0.589373,0.480497,0.253696,0.318536,-0.292422,0.038510,-0.453902,-0.062680,...,0.465049,-0.069603,-0.041326,-0.200051,-0.066358,0.210542,-0.334288,0.225605,-0.153953,-0.567041
4,0.485370,0.461014,-0.322709,-0.320388,0.581662,0.479701,-0.211599,0.605438,-0.223744,-0.197442,...,0.542439,0.017793,-0.102062,-0.629136,-0.114638,0.602666,-0.131385,0.432176,-0.717010,-0.257269
5,-0.360177,-0.136220,-0.272509,0.033101,0.142357,-0.518800,0.878265,-0.447239,-0.840521,-0.192055,...,0.698098,0.579451,0.616214,-0.780923,-0.153104,-0.527483,-0.625259,-0.915452,0.562845,-0.889039
6,-0.842615,-0.074463,0.796954,-0.809927,-0.880440,-0.486913,0.621741,-0.769738,0.809591,-0.520643,...,-0.005629,0.131275,0.862668,-0.280193,-0.753323,-0.945857,-0.391911,-0.798829,-0.097703,-0.464328
7,0.473839,-0.248154,0.425057,0.742996,-0.037749,0.550943,-0.421576,-0.647649,0.205312,-0.713291,...,0.634304,0.402907,0.455967,0.800453,-0.036726,-0.444707,0.595285,0.362282,0.167683,0.548073
8,-0.734781,-0.285714,0.513170,-0.207548,-0.188471,-0.330579,0.515846,-0.129989,0.773554,-0.593900,...,-0.140914,0.015674,0.506568,-0.443497,0.508552,-0.527223,0.430326,-0.240876,-0.393842,0.752347


In [36]:
# https://github.com/DMCB-GIST/SDGCCA/

import torch.nn as nn
import torch.nn.init as init
import torch
import random
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score, matthews_corrcoef

# Calculate performance metric
def calculate_metric(y_true, y_pred, y_pred_proba):
    acc = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    auc = roc_auc_score(y_true, y_pred_proba)
    mcc = matthews_corrcoef(y_true, y_pred)

    return acc, f1, auc, mcc

# Check mean and standard deviation
def check_mean_std_performance(result):
    return_list = []

    for m in ['ACC', 'F1', 'AUC', 'MCC']:
        return_list.append('{:.2f}+-{:.2f}'.format(np.array(result[m]).mean() * 100, np.array(result[m]).std() * 100))

    return return_list

# Setting random seed
def set_seed(random_seed):
    # Seed Setting
    random.seed(random_seed)
    np.random.seed(random_seed)
    torch.manual_seed(random_seed)
    torch.cuda.manual_seed(random_seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# For early stopping
class EarlyStopping:
    def __init__(self, patience=100, delta=1e-3):
        self.patience = patience
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta

    def __call__(self, val_loss):
        score = -val_loss

        if self.best_score is None:
            self.best_score = score

        elif score < self.best_score + self.delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

        elif score > self.best_score:
            self.best_score = score
            self.counter = 0
            self.early_stop = False

        else:
            self.best_score = score
            self.counter = 0

# Toy Dataset Class
"""
- Label: Binary
- Modality1: n(376) x d1 (18164)
- Modality1: n(376) x d2 (19353)
- Modality1: n(376) x d3 (309)
"""
class Toy_Dataset:
    def __init__(self, random_seed):
        # Make random dataset
        label = np.random.randint(2, size=(376, 1))
        data1 = np.random.rand(376, 18164)
        data2 = np.random.rand(376, 19353)
        data3 = np.random.rand(376, 309)

        # 5CV Dataset
        self.dataset = {'cv1': None, 'cv2': None, 'cv3': None, 'cv4': None, 'cv5': None}

        # Split Train,Validation and Test with 5 CV Fold
        kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_seed)
        for i, (train_val_index, test_index) in enumerate(kf.split(data1, label)):
            x_train_val_1, x_test_1, y_train_val, y_test = data1[train_val_index], data1[test_index], label[
                train_val_index], label[test_index]
            x_train_val_2, x_test_2 = data2[train_val_index], data2[test_index]
            x_train_val_3, x_test_3 = data3[train_val_index], data3[test_index]

            # Split Train and Validation
            x_train_1, x_val_1, y_train, y_val = train_test_split(x_train_val_1, y_train_val, test_size=0.2,
                                                                  random_state=random_seed)
            x_train_2, x_val_2, _, _ = train_test_split(x_train_val_2, y_train_val, test_size=0.2,
                                                        random_state=random_seed)
            x_train_3, x_val_3, _, _ = train_test_split(x_train_val_3, y_train_val, test_size=0.2,
                                                        random_state=random_seed)

            # CV Dataset
            cv_dataset = [[x_train_1, x_val_1, x_test_1], [x_train_2, x_val_2, x_test_2],
                          [x_train_3, x_val_3, x_test_3], [y_train, y_val, y_test]]
            self.dataset['cv' + str(i + 1)] = cv_dataset

    def __call__(self, cv, tensor=True, device=None):
        [x_train_1, x_val_1, x_test_1], [x_train_2, x_val_2, x_test_2], [x_train_3, x_val_3, x_test_3], \
        [y_train, y_val, y_test] = self.dataset['cv' + str(cv + 1)]

        # Numpy to tensor
        # Modality 1
        x_train_1 = torch.tensor(x_train_1).float().to(device)
        x_val_1 = torch.tensor(x_val_1).float().to(device)
        x_test_1 = torch.tensor(x_test_1).float().to(device)

        # Modality 2
        x_train_2 = torch.tensor(x_train_2).float().to(device)
        x_val_2 = torch.tensor(x_val_2).float().to(device)
        x_test_2 = torch.tensor(x_test_2).float().to(device)

        # Modality 3
        x_train_3 = torch.tensor(x_train_3).float().to(device)
        x_val_3 = torch.tensor(x_val_3).float().to(device)
        x_test_3 = torch.tensor(x_test_3).float().to(device)

        # Label
        y_train = torch.tensor(y_train).long().to(device)
        y_val = torch.tensor(y_val).long().to(device)
        y_test = torch.tensor(y_test).long().to(device)

        return [x_train_1, x_val_1, x_test_1], [x_train_2, x_val_2, x_test_2], [x_train_3, x_val_3, x_test_3], \
        [y_train, y_val, y_test]

# Weight & Bias Initialization
def initialization(net):
    if isinstance(net, nn.Linear):
        init.xavier_uniform(net.weight)
        init.zeros_(net.bias)

# SDGCCA with 3 Modality
class SDGCCA_3_M(nn.Module):
    def __init__(self, m1_embedding_list, m2_embedding_list, m3_embedding_list, top_k):
        super(SDGCCA_3_M, self).__init__()
        """
        m1_embedding_list = [d_1, ..., \bar{d_1}]
        m1_embedding_list = [d_2, ..., \bar{d_2}]
        m1_embedding_list = [d_3, ..., \bar{d_3}]
        top_k = k
        """

        # Embedding List of each modality
        m1_du0, m1_du1, m1_du2, m1_du3 = m1_embedding_list
        m2_du0, m2_du1, m2_du2, m2_du3 = m2_embedding_list
        m3_du0, m3_du1, m3_du2 = m3_embedding_list

        # Deep neural network of each modality
        self.model1 = nn.Sequential(
            nn.Linear(m1_du0, m1_du1), nn.Tanh(),
            nn.Linear(m1_du1, m1_du2), nn.Tanh(),
            nn.Linear(m1_du2, m1_du3), nn.Tanh())

        self.model2 = nn.Sequential(
            nn.Linear(m2_du0, m2_du1), nn.Tanh(),
            nn.Linear(m2_du1, m2_du2), nn.Tanh(),
            nn.Linear(m2_du2, m2_du3), nn.Tanh())

        self.model3 = nn.Sequential(
            nn.Linear(m3_du0, m3_du1), nn.Tanh(),
            nn.Linear(m3_du1, m3_du2), nn.Tanh())

        # Weight & Bias Initialization
        self.model1.apply(initialization)
        self.model2.apply(initialization)
        self.model3.apply(initialization)

        self.top_k = top_k

        # Projection matrix
        self.U = None

        # Softmax Function
        self.softmax = nn.Softmax(dim=1)

    # Input: Each modality
    # Output: Deep neural network output of the each modality = [H_1, H_2, H_3]
    def forward(self, x1, x2, x3):
        output1 = self.model1(x1)
        output2 = self.model2(x2)
        output3 = self.model3(x3)

        return output1, output2, output3

    # Calculate correlation loss
    def cal_loss(self, H_list, train=True):
        eps = 1e-8
        AT_list = []

        for H in H_list:
            assert torch.isnan(H).sum().item() == 0
            m = H.size(1)  # out_dim
            Hbar = H - H.mean(dim=1).repeat(m, 1).view(-1, m)
            assert torch.isnan(Hbar).sum().item() == 0

            A, S, B = Hbar.svd(some=True, compute_uv=True)
            A = A[:, :self.top_k]
            assert torch.isnan(A).sum().item() == 0

            S_thin = S[:self.top_k]
            S2_inv = 1. / (torch.mul(S_thin, S_thin) + eps)
            assert torch.isnan(S2_inv).sum().item() == 0

            T2 = torch.mul(torch.mul(S_thin, S2_inv), S_thin)
            assert torch.isnan(T2).sum().item() == 0

            T2 = torch.where(T2 > eps, T2, (torch.ones(T2.shape) * eps).to(H.device))
            T = torch.diag(torch.sqrt(T2))
            assert torch.isnan(T).sum().item() == 0

            T_unnorm = torch.diag(S_thin + eps)
            assert torch.isnan(T_unnorm).sum().item() == 0

            AT = torch.mm(A, T)
            AT_list.append(AT)

        M_tilde = torch.cat(AT_list, dim=1)
        assert torch.isnan(M_tilde).sum().item() == 0

        Q, R = M_tilde.qr()
        assert torch.isnan(R).sum().item() == 0
        assert torch.isnan(Q).sum().item() == 0

        U, lbda, _ = R.svd(some=False, compute_uv=True)
        assert torch.isnan(U).sum().item() == 0
        assert torch.isnan(lbda).sum().item() == 0

        G = Q.mm(U[:, :self.top_k])
        assert torch.isnan(G).sum().item() == 0

        U = []  # Projection Matrix

        # Get mapping to shared space
        views = H_list
        F = [H.shape[0] for H in H_list]  # features per view
        for idx, (f, view) in enumerate(zip(F, views)):
            _, R = torch.qr(view)
            Cjj_inv = torch.inverse((R.T.mm(R) + eps * torch.eye(view.shape[1], device=view.device)))
            assert torch.isnan(Cjj_inv).sum().item() == 0
            pinv = Cjj_inv.mm(view.T)

            U.append(pinv.mm(G))

        # If model training -> Change projection matrix
        # Else -> Using projection matrix for calculate correlation loss
        if train:
            self.U = U
            for i in range(len(self.U)):
                self.U[i] = nn.Parameter(torch.tensor(self.U[i]))

        _, S, _ = M_tilde.svd(some=True)

        assert torch.isnan(S).sum().item() == 0
        use_all_singular_values = False
        if not use_all_singular_values:
            S = S.topk(self.top_k)[0]
        corr = torch.sum(S)
        assert torch.isnan(corr).item() == 0

        loss = - corr
        return loss

    # SDGCCA prediction
    # Input: Each modality
    # Output: Soft voting of the label presentation of each modality
    def predict(self, x1, x2, x3):
        out1 = self.model1(x1)
        out2 = self.model2(x2)
        out3 = self.model3(x3)

        t1 = torch.matmul(out1, self.U[0])
        t2 = torch.matmul(out2, self.U[1])
        t3 = torch.matmul(out3, self.U[2])

        y_hat1 = torch.matmul(t1, torch.pinverse(self.U[3]))
        y_hat2 = torch.matmul(t2, torch.pinverse(self.U[3]))
        y_hat3 = torch.matmul(t3, torch.pinverse(self.U[3]))
        y_ensemble = (y_hat1+y_hat2+y_hat3)/3

        y_hat1 = self.softmax(y_hat1)
        y_hat2 = self.softmax(y_hat2)
        y_hat3 = self.softmax(y_hat3)
        y_ensemble = self.softmax(y_ensemble)

        return y_hat1, y_hat2, y_hat3, y_ensemble

In [42]:
from tqdm.auto import tqdm
import warnings
warnings.simplefilter("ignore", UserWarning)

# Seed Setting
random_seed = 100
set_seed(random_seed)

def train_SDGCCA(hyper_dict):
    # Return List
    ensemble_list = {'ACC': [], 'F1': [], 'AUC': [], 'MCC': []}
    metric_list = ['ACC', 'F1', 'AUC', 'MCC']
    hyper_param_list = []
    best_hyper_param_list = []
    
    # Prepare Toy Dataset
    dataset = Toy_Dataset(hyper_dict['random_seed'])

    # 5 CV
    for cv in tqdm(range(5), desc='CV...'):
        # Prepare Dataset
        [x_train_1, x_val_1, x_test_1], [x_train_2, x_val_2, x_test_2], [x_train_3, x_val_3, x_test_3], \
        [y_train, y_val, y_test] = dataset(cv, tensor=True, device=hyper_dict['device'])
        
        # Define Deep neural network dimension of the each modality
        m1_embedding_list = [x_train_1.shape[1]] + hyper_dict['embedding_size']
        m2_embedding_list = [x_train_2.shape[1]] + hyper_dict['embedding_size']
        m3_embedding_list = [x_train_3.shape[1]] + hyper_dict['embedding_size'][1:]

        # Train Label -> One_Hot_Encoding
        y_train_onehot = torch.zeros(y_train.shape[0], 2).float().to(hyper_dict['device'])
        y_train_onehot[range(y_train.shape[0]), y_train.squeeze()] = 1

        # Find Best K by Validation MCC
        val_mcc_result_list = []
        test_ensemble_dict = {'ACC': [], 'F1': [], 'AUC': [], 'MCC': []}

        # Grid search for find best hyperparameter by Validation MCC
        for top_k in tqdm(range(1, hyper_dict['max_top_k']+1), desc='Grid seach for find best hyperparameter...'):
            for lr in hyper_dict['lr']:
                for reg in hyper_dict['reg']:
                    hyper_param_list.append([top_k, lr, reg])
                    early_stopping = EarlyStopping(patience=hyper_dict['patience'], delta=hyper_dict['delta'])
                    best_loss = np.Inf

                    # Define SDGCCA with 3 modality
                    model = SDGCCA_3_M(m1_embedding_list, m2_embedding_list, m3_embedding_list, top_k).to(hyper_dict['device'])

                    # Optimizer
                    clf_optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=reg)

                    # Cross Entropy Loss
                    criterion = nn.CrossEntropyLoss()

                    # Model Train
                    for i in range(hyper_dict['epoch']):
                        model.train()

                        # Calculate correlation loss
                        out1, out2, out3 = model(x_train_1, x_train_2, x_train_3)
                        cor_loss = model.cal_loss([out1, out2, out3, y_train_onehot])

                        # Calculate classification loss
                        clf_optimizer.zero_grad()

                        y_hat1, y_hat2, y_hat3, _ = model.predict(x_train_1, x_train_2, x_train_3)
                        clf_loss1 = criterion(y_hat1, y_train.squeeze())
                        clf_loss2 = criterion(y_hat2, y_train.squeeze())
                        clf_loss3 = criterion(y_hat3, y_train.squeeze())

                        clf_loss = clf_loss1 + clf_loss2 + clf_loss3

                        clf_loss.backward()
                        clf_optimizer.step()

                        # Model Validation
                        with torch.no_grad():
                            model.eval()
                            _, _, _, y_ensemble = model.predict(x_val_1, x_val_2, x_val_3)
                            val_loss = criterion(y_ensemble, y_val.squeeze())

                            early_stopping(val_loss)
                            if val_loss < best_loss:
                                best_loss = val_loss

                            if early_stopping.early_stop:
                                break

                    # Load Best Model
                    model.eval()

                    # Model Validation
                    _, _, _, ensembel_y_hat = model.predict(x_val_1, x_val_2, x_val_3)
                    y_pred_ensemble = torch.argmax(ensembel_y_hat, 1).cpu().detach().numpy()
                    y_pred_proba_ensemble = ensembel_y_hat[:, 1].cpu().detach().numpy()            
                    _, _, _, val_mcc = calculate_metric(y_val.cpu().detach().numpy(), y_pred_ensemble, y_pred_proba_ensemble)
                    val_mcc_result_list.append(val_mcc)

                    # Model Tset
                    _, _, _, ensembel_y_hat = model.predict(x_test_1, x_test_2, x_test_3)
                    y_pred_ensemble = torch.argmax(ensembel_y_hat, 1).cpu().detach().numpy()
                    y_pred_proba_ensemble = ensembel_y_hat[:, 1].cpu().detach().numpy()
                    test_acc, test_f1, test_auc, test_mcc = calculate_metric(y_test.cpu().detach().numpy(), y_pred_ensemble, y_pred_proba_ensemble)
                    ensemble_result = [test_acc, test_f1, test_auc, test_mcc]
                    for k, metric in enumerate(metric_list):
                        test_ensemble_dict[metric].append(ensemble_result[k])

        # Find best K
        best_k = np.argmax(val_mcc_result_list)
        
        # Find best hyperparameter
        best_hyper_param_list.append(hyper_param_list[best_k])
        
        # Append Best K Test Result
        for metric in metric_list:
            ensemble_list[metric].append(test_ensemble_dict[metric][best_k])

    return ensemble_list, best_hyper_param_list

hyper_dict = {'epoch': 1000, 'delta': 0, 'random_seed': random_seed,
              'device': torch.device("cuda:1" if torch.cuda.is_available() else "cpu"),
              'lr': [0.0001,0.00001], 'reg': [0, 0.01,0.0001], 
              'patience': 30, 'embedding_size': [256, 64, 16], 'max_top_k': 10}

ensemble_list, hyper = train_SDGCCA(hyper_dict)

CV...:   0%|          | 0/5 [00:00<?, ?it/s]

Grid seach for find best hyperparameter...:   0%|          | 0/10 [00:00<?, ?it/s]

Grid seach for find best hyperparameter...:   0%|          | 0/10 [00:00<?, ?it/s]

Grid seach for find best hyperparameter...:   0%|          | 0/10 [00:00<?, ?it/s]

Grid seach for find best hyperparameter...:   0%|          | 0/10 [00:00<?, ?it/s]

Grid seach for find best hyperparameter...:   0%|          | 0/10 [00:00<?, ?it/s]

In [41]:
# ensemble_list, hyper = train_SDGCCA(hyper_dict)

# Check Performance
performance_result = check_mean_std_performance(ensemble_list)

print('Test Performance')
print('ACC: {} F1: {} AUC: {} MCC: {}'.format(performance_result[0], performance_result[1], performance_result[2], performance_result[3]))

print('\nBest Hyperparameter')
for i, h in enumerate(hyper):
    print('CV: {} Best k: {} Learning Rage: {} Regularization Term: {}'.format(i+1, h[0], h[1], h[2]))

NameError: ignored